In [1]:
%%capture
%%bash
. /opt/conda/bin/activate
conda install -c bioconda -y samtools
conda install -c bioconda -y tabix
conda install -c bioconda -y pbmm2 pbsv

In [2]:
%%capture
%%bash
dx download -f --recursive /Haplotype_ASM_0525_2019/
ln -sf Haplotype_ASM_0525_2019/* .

In [3]:
%%bash
dx download -f /jc_notebook/MHC_37.fa 

In [4]:
%%capture
%%bash
git clone https://github.com/lh3/minimap2
cd minimap2 && make

In [5]:
%%capture
%%bash
curl -L https://github.com/attractivechaos/k8/releases/download/v0.2.4/k8-0.2.4.tar.bz2 | tar -jxf -
cp k8-0.2.4/k8-`uname -s` k8   # or copy it to a directory on you $PATH

In [6]:
%%bash
cp MHC_37.fa ref.fa

In [7]:
%%bash
minimap2/minimap2 -c --cs ref.fa p_ctg_cns_H1.fa | \
sort -k6,6 -k8,8n  | \
./k8 minimap2/misc/paftools.js call -f ref.fa  -L200 - > H1toMHC_var.vcf
bgzip -f H1toMHC_var.vcf
tabix -p vcf -f H1toMHC_var.vcf.gz

[M::mm_idx_gen::0.156*1.00] collected minimizers
[M::mm_idx_gen::0.192*1.33] sorted minimizers
[M::main::0.192*1.33] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.206*1.30] mid_occ = 99
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.216*1.28] distinct minimizers: 768040 (92.97% are singletons); average occurrences: 1.219; average spacing: 5.307
[M::worker_pipeline::12.086*1.04] mapped 7 sequences
[M::main] Version: 2.17-r954-dirty
[M::main] CMD: minimap2/minimap2 -c --cs ref.fa p_ctg_cns_H1.fa
[M::main] Real time: 12.095 sec; CPU: 12.564 sec; Peak RSS: 0.239 GB
4861902 reference bases covered by exactly one contig
16457 substitutions; ts/tv = 2.119
625 1bp deletions
537 1bp insertions
262 2bp deletions
230 2bp insertions
451 [3,50) deletions
399 [3,50) insertions
24 [50,1000) deletions
23 [50,1000) insertions
3 >=1000 deletions
1 >=1000 insertions


In [8]:
%%bash
minimap2/minimap2 -c --cs ref.fa p_ctg_cns_H2.fa | \
sort -k6,6 -k8,8n  | \
./k8 minimap2/misc/paftools.js call -f ref.fa  -L200 - > H2toMHC_var.vcf
bgzip -f H2toMHC_var.vcf
tabix -p vcf -f H2toMHC_var.vcf.gz

[M::mm_idx_gen::0.155*0.98] collected minimizers
[M::mm_idx_gen::0.194*1.38] sorted minimizers
[M::main::0.194*1.38] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.208*1.37] mid_occ = 99
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.217*1.34] distinct minimizers: 768040 (92.97% are singletons); average occurrences: 1.219; average spacing: 5.307
[M::worker_pipeline::12.703*1.02] mapped 10 sequences
[M::main] Version: 2.17-r954-dirty
[M::main] CMD: minimap2/minimap2 -c --cs ref.fa p_ctg_cns_H2.fa
[M::main] Real time: 12.712 sec; CPU: 13.004 sec; Peak RSS: 0.206 GB
4847120 reference bases covered by exactly one contig
17832 substitutions; ts/tv = 2.152
583 1bp deletions
598 1bp insertions
226 2bp deletions
231 2bp insertions
437 [3,50) deletions
445 [3,50) insertions
24 [50,1000) deletions
32 [50,1000) insertions
5 >=1000 deletions
3 >=1000 insertions


In [9]:
%%bash
dx download -f --recursive /phased_reads_2/

In [10]:
%%bash
git clone https://github.com/lh3/seqtk.git
cd seqtk; make

gcc -g -Wall -O2 -Wno-unused-function seqtk.c -o seqtk -lz -lm


Cloning into 'seqtk'...
seqtk.c: In function ‘stk_comp’:
seqtk.c:399:16: warning: variable ‘lc’ set but not used [-Wunused-but-set-variable]
    int la, lb, lc, na, nb, nc, cnt[11];
                ^~


In [11]:
%%bash
zcat phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.fastq.gz | awk 'NR % 4 ==1' | tr -d "@" | sort > ids1
cat phased_reads_2/asm_mapped.fa | grep ">" | tr -d ">" | sort > ids2
comm  -13 ids1 ids2  > unphased_reads_on_assembly_ids
./seqtk/seqtk subseq phased_reads_2/asm_mapped.fa unphased_reads_on_assembly_ids > unphased_reads_on_assembly.fa

In [12]:
%%bash
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.H1.fastq.gz > reads_H1.fa
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.untagged.fastq.gz >> reads_H1.fa
cat unphased_reads_on_assembly.fa >> reads_H1.fa

In [13]:
%%bash
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.H2.fastq.gz > reads_H2.fa
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.untagged.fastq.gz >> reads_H2.fa
cat unphased_reads_on_assembly.fa >> reads_H2.fa

In [14]:
%%bash
minimap2/minimap2 -a -x asm5 ref.fa reads_H1.fa | samtools view -bS - | samtools sort - > reads_H1_MHC.bam 
samtools index reads_H1_MHC.bam
minimap2/minimap2 -a -x asm5 ref.fa reads_H2.fa | samtools view -bS - | samtools sort - > reads_H2_MHC.bam 
samtools index reads_H2_MHC.bam

[M::mm_idx_gen::0.124*1.00] collected minimizers
[M::mm_idx_gen::0.143*1.23] sorted minimizers
[M::main::0.143*1.23] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.151*1.22] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.156*1.23] distinct minimizers: 438505 (95.36% are singletons); average occurrences: 1.139; average spacing: 9.953
[M::worker_pipeline::22.801*2.67] mapped 14990 sequences
[M::main] Version: 2.17-r954-dirty
[M::main] CMD: minimap2/minimap2 -a -x asm5 ref.fa reads_H1.fa
[M::main] Real time: 22.815 sec; CPU: 60.932 sec; Peak RSS: 0.421 GB
[M::mm_idx_gen::0.125*0.96] collected minimizers
[M::mm_idx_gen::0.145*1.24] sorted minimizers
[M::main::0.145*1.24] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.153*1.23] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.158*1.21] distinct minimizers: 438505 (95.36% are singletons); average occur

In [15]:
%%bash
pbmm2 align ref.fa reads_H1.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:movie1' reads_H1_MHC.bam
pbmm2 align ref.fa reads_H2.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:movie1' reads_H2_MHC.bam

>|> 20190903 19:56:59.007 -|- WARN -|- CheckPositionalArgs -|- 0x7fa6ce8e3b80|| -|- Input is FASTA. Output BAM file cannot be used for polishing with GenomicConsensus!
>|> 20190903 19:56:59.007 -|- WARN -|- CheckPositionalArgs -|- 0x7fa6ce8e3b80|| -|- Warning: Overwriting existing output file: reads_H1_MHC.bam
>|> 20190903 19:57:17.447 -|- WARN -|- CheckPositionalArgs -|- 0x7eff4e1deb80|| -|- Input is FASTA. Output BAM file cannot be used for polishing with GenomicConsensus!
>|> 20190903 19:57:17.447 -|- WARN -|- CheckPositionalArgs -|- 0x7eff4e1deb80|| -|- Warning: Overwriting existing output file: reads_H2_MHC.bam


In [16]:
%%bash
samtools faidx ref.fa
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload ref.fa*

ID                  file-FbgGFG80Xv397KfJKb6j3xf2
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                ref.fa
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:57:37 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:57:37 2019
Media type          
archivalState       "live"
ID                  file-FbgGFG80Xv3531ZP2jjf4Jq3
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                ref.fa.fai
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:57:37 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk8

In [17]:
%%bash
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload reads_H2_MHC.bam* reads_H1_MHC.bam* H2toMHC_var.vcf* H1toMHC_var.vcf*

ID                  file-FbgGFGj0Xv35YYZv2jY144bP
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H2_MHC.bam
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:57:39 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:57:40 2019
Media type          
archivalState       "live"
ID                  file-FbgGFJ00Xv3KVp084QQZyyyy
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H2_MHC.bam.bai
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:57:40 2019
Created by          jchin_dx
 via the job        job-

In [18]:
%%bash
pbsv discover reads_H1_MHC.bam reads_H1_MHC.svsig.gz 
pbsv discover reads_H2_MHC.bam reads_H2_MHC.svsig.gz 

In [19]:
%%bash
pbsv call --ccs ref.fa reads_H1_MHC.svsig.gz reads_H1_MHC.var.vcf
pbsv call --ccs ref.fa reads_H2_MHC.svsig.gz reads_H2_MHC.var.vcf
bgzip -f reads_H1_MHC.var.vcf 
bgzip -f reads_H2_MHC.var.vcf
tabix -p vcf -f reads_H1_MHC.var.vcf.gz
tabix -p vcf -f reads_H2_MHC.var.vcf.gz

In [20]:
%%bash
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload reads_H1_MHC.var.vcf* reads_H2_MHC.var.vcf*

ID                  file-FbgGFf80Xv3077q6Kb9VByZJ
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H1_MHC.var.vcf.gz
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:58:17 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:58:17 2019
Media type          
archivalState       "live"
ID                  file-FbgGFf80Xv3Jy2byKbPJ4yVp
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H1_MHC.var.vcf.gz.tbi
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:58:17 2019
Created by          jchin_dx
 via the j

In [21]:
%%bash

pbmm2 align p_ctg_cns_H1.fa reads_H1.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:H1' reads_H1.bam
pbmm2 align p_ctg_cns_H2.fa reads_H2.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:H2' reads_H2.bam
pbsv discover reads_H1.bam reads_H1.svsig.gz 
pbsv discover reads_H2.bam reads_H2.svsig.gz 
pbsv call --ccs p_ctg_cns_H1.fa reads_H1.svsig.gz reads_H1.var.vcf
pbsv call --ccs p_ctg_cns_H2.fa reads_H2.svsig.gz reads_H2.var.vcf
bgzip -f reads_H1.var.vcf 
bgzip -f reads_H2.var.vcf
tabix -p vcf -f reads_H1.var.vcf.gz
tabix -p vcf -f reads_H2.var.vcf.gz

>|> 20190903 19:58:19.580 -|- WARN -|- CheckPositionalArgs -|- 0x7f3a9c3d6b80|| -|- Input is FASTA. Output BAM file cannot be used for polishing with GenomicConsensus!
>|> 20190903 19:58:34.963 -|- WARN -|- CheckPositionalArgs -|- 0x7f988a411b80|| -|- Input is FASTA. Output BAM file cannot be used for polishing with GenomicConsensus!


In [22]:
%%bash
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload reads_H?.var.vcf.gz* reads_H?.bam* p_ctg_cns_H?.fa

ID                  file-FbgGFzQ0Xv3Kbk44Kf327Xk3
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H1.var.vcf.gz
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:58:54 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:58:54 2019
Media type          
archivalState       "live"
ID                  file-FbgGFzQ0Xv3Kbk44Kf327Xk5
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                reads_H1.var.vcf.gz.tbi
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:58:54 2019
Created by          jchin_dx
 via the job      

In [23]:
%%writefile readme.txt
#assembly to assembly SV call by pfatools
H1toMHC_var.vcf.gz
H1toMHC_var.vcf.gz.tbi
H2toMHC_var.vcf.gz
H2toMHC_var.vcf.gz.tbi

#notebook on how the variants are called
MHC_Analysis_00.ipynb

# assembly used
p_ctg_cns_H1.fa
p_ctg_cns_H2.fa

#this file
readme.txt

#reads to assembly variant call
reads_H1.bam
reads_H1.bam.bai
reads_H1.var.vcf.gz
reads_H1.var.vcf.gz.tbi
reads_H2.bam
reads_H2.bam.bai
reads_H2.var.vcf.gz
reads_H2.var.vcf.gz.tbi

#reads to GRCh37 MHC variant call
reads_H1_MHC.bam
reads_H1_MHC.bam.bai
reads_H1_MHC.var.vcf.gz
reads_H1_MHC.var.vcf.gz.tbi
reads_H2_MHC.bam
reads_H2_MHC.bam.bai
reads_H2_MHC.var.vcf.gz
reads_H2_MHC.var.vcf.gz.tbi

#GRCh 37 MHC seqeunce
ref.fa
ref.fa.fai

Writing readme.txt


In [24]:
%%bash
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload readme.txt

ID                  file-FbgGG1j0Xv3GYbJJ2k2Bp3X7
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                readme.txt
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:59:03 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:59:03 2019
Media type          
archivalState       "live"


In [25]:
%%bash
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.H1.fastq.gz > all_reads.fa
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.H2.fastq.gz > all_reads.fa
seqtk/seqtk seq -A phased_reads_2/HG002.Sequel.15kb.pbmm2.hs37d5.whatshap.haplotag.RTG.10x.trio.MHConly.28498559.untagged.fastq.gz >> all_reads.fa
cat unphased_reads_on_assembly.fa >> rall_reads.fa


pbmm2 align ref.fa all_reads.fa --sort --preset CCS --sample sample1 --rg '@RG\tID:movie1' all_CCS_reads_MHC.bam

pbsv discover all_CCS_reads_MHC.bam all_CCS_reads_MHC.svsig.gz 
pbsv call --ccs MHC_37.fa all_CCS_reads_MHC.svsig.gz all_CCS_reads_MHC.var.vcf
bgzip -f all_CCS_reads_MHC.var.vcf
tabix -p vcf -f all_CCS_reads_MHC.var.vcf.gz


>|> 20190903 19:59:06.997 -|- WARN -|- CheckPositionalArgs -|- 0x7f5437893b80|| -|- Input is FASTA. Output BAM file cannot be used for polishing with GenomicConsensus!


In [26]:
%%bash
dx mkdir -p /tmp_data_2
dx cd /tmp_data_2
dx upload all_CCS_reads_MHC.var.vcf.gz all_CCS_reads_MHC.var.vcf.gz.tbi  all_CCS_reads_MHC.bam  all_CCS_reads_MHC.bam.bai

ID                  file-FbgGGB80Xv3Kjz2P4Pj52b3J
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                all_CCS_reads_MHC.var.vcf.gz
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:59:37 2019
Created by          jchin_dx
 via the job        job-Fbg8Jk80Xv3F7zbyKvk80ZJp
Last modified       Tue Sep  3 12:59:37 2019
Media type          
archivalState       "live"
ID                  file-FbgGGB80Xv3KVp084QQZyz0K
Class               file
Project             project-FXGVFb00Xv39f6K624b11bZ1
Folder              /tmp_data_2
Name                all_CCS_reads_MHC.var.vcf.gz.tbi
State               closing
Visibility          visible
Types               -
Properties          -
Tags                -
Outgoing links      -
Created             Tue Sep  3 12:59:37 2019
Created by          jchin_dx
